In [ ]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install flickrapi


In [ ]:
from flickrapi import FlickrAPI
import pandas as pd
#import sys



In [ ]:
#set "key" and "secret" in a local configuration file named flickr_APIkey.txt
#for an example see: flickr_APIkey_example.txt
#read-in key and secret from configuration file
import os
os.chdir("/Users/KGthatsme/Analyses/FlickrFish")
file=open(file="flickr_APIkey.txt", mode="r") #mode r = read the file
key=file.readline()
key=key.rstrip() # Remove all trailing whitespace (e.g., \n)
secret=file.readline()
secret=secret.rstrip()

In [ ]:
def get_urls(image_tag, MAX_COUNT):    
    flickr = FlickrAPI(key, secret)
    count=0
    urls=[]
    uploads=[]
    geos=[]
    takens=[]
    for photo in flickr.walk(#text=image_tag, # Temporarily remove image_tag parameter, to see if I can get any geo information
                         #tag_mode='all',
                         #tags=image_tag, #use text parameter instead, which searches title, description, and tags
                         #bbox='-71.791037, 41.217233, -71.095148, 42.007849', # Narraganset Bay + RI Sound
                         has_geo=1,
                         #lat=41.604168,
                         #lon=-71.320683,
                         extras='url_o, date_upload, date_taken, geo',
                         per_page=MAX_COUNT,
                         sort='relevance'
                        ):
        if count< MAX_COUNT:
            count=count+1
            print("Fetching data for image number {}".format(count))
            try:
                url=photo.get('url_o')
                print("URL: {}".format(url))
                urls.append(url)
                upload=photo.get('date_upload')
                print("Upload: {}".format(upload))
                uploads.append(upload)
                taken=photo.get('date_taken')
                print("Taken: {}".format(taken))
                takens.append(taken)
                geo=photo.get('geo')
                print("Geo: {}".format(geo))
                geos.append(geo)
            except:
                print("Url for image number {} could not be fetched".format(count))
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls),MAX_COUNT))
            break
    urls=pd.Series(urls)
    uploads=pd.Series(uploads)
    geos=pd.Series(geos)
    takens=pd.Series(takens)
    print("Writing out the urls in the current directory")
    urls.to_csv(image_tag+"_urls.csv")
    uploads.to_csv(image_tag+"_uploads.csv")
    takens.to_csv(image_tag+"_takens.csv")
    geos.to_csv(image_tag+"_geos.csv")
    print("Done!!!")



In [ ]:
# change wd to RESULTS folder
os.chdir("/Users/KGthatsme/Analyses/FlickrFish/RESULTS")

In [ ]:
image_tag="fish"
MAX_COUNT=100
get_urls(image_tag=image_tag, MAX_COUNT=MAX_COUNT)

In [ ]:
## Only need the following code if running this from Terminal
#def main(): 
#    tag=sys.argv[1]
#    MAX_COUNT=int(sys.argv[2])
#    get_urls(tag,MAX_COUNT)


#if __name__=='__main__': 
#    main() 
# if running the module as the main program (e.g., in terminal: python getFishDat.py): execute function (main)
# if module is imported by another program, _name_ is assigned to the module name "getFishDat": skip function (main)

import csv
import requests
import os
import sys
import time
def put_images(image_tag):
    urls=[]
    FILE_NAME=image_tag+"_urls.csv"
    with open(FILE_NAME,newline="") as csvfile:
        doc=csv.reader(csvfile,delimiter=",")
        for row in doc:
            if row[1].startswith("https"):
                urls.append(row[1])
    if not os.path.isdir(os.path.join(os.getcwd(),FILE_NAME.split("_")[0])):
        os.mkdir(FILE_NAME.split("_")[0])
    t0=time.time()
    for url in enumerate(urls):
        print("Starting download {} of ".format(url[0]+1),len(urls))
        try:
            resp=requests.get(url[1],stream=True)
            path_to_write=os.path.join(os.getcwd(),FILE_NAME.split("_")[0],url[1].split("/")[-1])
            outfile=open(path_to_write,'wb')
            outfile.write(resp.content)
            outfile.close()
            print("Done downloading {} of {}".format(url[0]+1,len(urls)))
        except:
            print("Failed to download url number {}".format(url[0]))
    t1=time.time()
    print("Done with download, job took {} seconds".format(t1-t0))





In [ ]:
put_images(image_tag=image_tag)